---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [2]:
def answer_one():
        
    # Your Code Here
    Gmovie = nx.Graph()                     # create the graph
    Gmovie.add_nodes_from(employees, bipartite=0)    # add bipartite nodes section 0
    Gmovie.add_nodes_from(movies, bipartite=1)      # add bipartite nodes section 1
    movie_edges = nx.read_edgelist('Employee_Movie_Choices.txt', delimiter='\t')   # import the edgelist as tuples
    Gmovie.add_edges_from(movie_edges.edges(data=True))    #add edges to the graph 

    return Gmovie

# answer_one()

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [3]:
def answer_two():
    
    # Your Code Here
    G = answer_one()
    
    for employee in employees:
        G.node[employee]['type'] = 'employee'
        
    for movie in movies:
        G.node[movie]['type'] = 'movie'

    return G 

# answer_two()

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [4]:
def answer_three():
        
    # Your Code Here
    G = answer_two()
    P = bipartite.weighted_projected_graph(G, employees)
    
    return P  # .edges(data=True)

# answer_three()

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [6]:
def answer_four():
        
    # Your Code Here
    ans_three = answer_three()  # use answer from #3
    
    # turn above into a dataframe
    edge_data1 = pd.DataFrame(ans_three.edges(data=True), columns=['name1', 'name2', 'weight'])
    edge_data1['weight'] = edge_data1['weight'].map(lambda x: x['weight'])  # extract the weight attribute

    edge_data2 = edge_data1.reindex_axis(['name2', 'name1', 'weight'], axis=1)  # reindex to change the column order new df
    edge_data2.columns = ['name1', 'name2', 'weight']   # rename columns to be the same as edge_data1

    edge_df = pd.concat([edge_data1, edge_data2])  # concatanate the two dataframes together
    
    # read in the relationship data 
    relation_df = pd.read_csv('Employee_Relationships.txt', delim_whitespace=True, 
                          header=None, names=['name1', 'name2', 'relationship'])
    # merge the edge data from above with the relationship data and clean it up
    new = relation_df.merge(edge_df, how='left', on=['name1', 'name2']).drop_duplicates().reset_index(drop=True).fillna(0)
    correlation = new.corr()['weight'][0]  # find the correlation 

    return correlation

# answer_four()

0.78839622217334759